In [18]:
# load ASE atom from POSCAR file
from ase.io.vasp import read_vasp
import matplotlib.pyplot as plt
import os
import numpy as np
from glob import glob
import pickle

from ase import Atoms
from ase.neighborlist import NeighborList
from ase.build import make_supercell

from ovito.modifiers import CoordinationAnalysisModifier
from ovito.pipeline import StaticSource, Pipeline
from ovito.io.ase import ase_to_ovito

from mlptools.data.structures import generate_crystal
from mlptools.io.write import QuantumEspressoWriter

In [19]:
# randomly displace atoms while compressing or expanding the cell
path2root = '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/stishovite__mp-6947'
path2POSCAR = os.path.join(path2root, "original", "POSCAR")
path2save = os.path.join(path2root, "relax")
path2input = os.path.join(path2root, "input")

original_atoms = read_vasp(path2POSCAR)

supercell_atoms = make_supercell(original_atoms, [[2,0,0],[0,2,0],[0,0,2]])

print(f"Number of atoms in original: {len(original_atoms)}")
print(f"Number of atoms in supercell: {len(supercell_atoms)}")

Number of atoms in original: 6
Number of atoms in supercell: 48


In [17]:
with open(os.path.join(path2input, "relax.in")) as f:
    original_scf_input_lines = [s.strip() for s in f.readlines()]

for i, atoms in enumerate([supercell_atoms]):
    writer = QuantumEspressoWriter(atoms, path2input, scf_filename="relax.in")
    scf_input_lines = writer.output()

    # create dir
    path2result = f'{path2save}/relax_{i}'
    if not os.path.exists(path2result):
        os.makedirs(path2result)
    else:
        print(f'{path2result} already exists')

    # write scf.in
    with open(f'{path2result}/relax.in', 'w') as f:
        f.writelines('\n'.join(scf_input_lines))